# Лабораторная работа 1

In [ ]:
# ответ на 1 гипотезу из примера
# df.pivot_table(index='City',colums='Day',values='userID', agg='count')

# Цель
Проверить 3 гипотезы:

1. Период нулевых годов 21-го века является самым прибыльным для фильмов
2. Не существует таких жанров в США, где сумма лайков в фейсбуке 1ых актёров меньше суммы лайков 2ых или 3их актёров 
3. Самые прибыльные жанры в США и в остальном мире совпадают

In [1]:
import pandas as pd
import numpy as np

# открытие файла
df = pd.read_csv('Dataset1.csv')

# Этап 1 - Обзор данных

### Выводим первые 5 строк таблицы

In [2]:
df.head()

,color,Director_Name,num_Critic_for_reviews,duration,director_Facebook_likes,actor_3_Facebook_likes,actor_2_name,Actor_1_Facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes;
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000;
1,Colour,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0;
2,Colour,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000;
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000;
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0;


### Получаем информацию о таблице

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   color                       5025 non-null   object 
 1   Director_Name               4872 non-null   object 
 2   num_Critic_for_reviews      4927 non-null   float64
 3   duration                    4959 non-null   float64
 4   director_Facebook_likes     4872 non-null   float64
 5   actor_3_Facebook_likes      4953 non-null   float64
 6   actor_2_name                4963 non-null   object 
 7   Actor_1_Facebook_likes      4968 non-null   float64
 8   gross                       4104 non-null   float64
 9   genres                      4974 non-null   object 
 10  actor_1_name                4968 non-null   object 
 11  movie_Title                 4974 non-null   object 
 12  num_voted_users             4974 non-null   float64
 13   cast_total_facebook_likes  4974 

### Выводы
- в таблице 28 колонок и 5025 строк
- так как столбцы содержат разное количество ненулевых строк, то есть пропущенные значения
- данных достаточно для проверки гипотез

# Этап 2 - Обработка данных

In [4]:
# Выводим количество дубликатов
df.duplicated().sum()

45

In [5]:
# Удаляем дубликаты с обновлением индекса
df = df.drop_duplicates().reset_index(drop=True)
# Выводим их количество
df.duplicated().sum()

0

При обзоре данных мы заметили, что в колонке 'color' в некоторых местах есть ячейки, которые содержат в себе информацию всей строки таблицы. Поэтому нужно это исправить

In [6]:
# заменяем пропущенные значения в колонке 'color'
df['color'] = df['color'].fillna('Color')

# функция для удаления запятых в названиях фильмов
def delete_comma_in_title(line: str) -> str:
    # что делает код:
    # 1) разделяет по двойным ковычкам. пр: aaaa,"b, b, bb",ccc
    # 2) берёт средний элемент - название фильма, и удаляет запятые
    # 3) превращает список обратно в строку. пр: aaaa,b b bb,ccc
    line = line.split('"')
    line[1] = line[1].replace(',','')
    return ''.join(line)

# находим ячейки, которые содержат в себе информацию всей строки таблицы
color = df['color']

# проходимся по столбцу 'color'
for i in range(len(color)) :
    # если ячейка при разбиении по запятым содержит не только слово 'color'
    if len(color[i].split(',')) > 1:
        # удаляем запятые в названии
        color[i] = delete_comma_in_title(color[i])
        # заменяем строку датафрейма
        line = pd.Series(color[i].split(','))
        df.iloc[i] = line

C:\Users\Azerty\AppData\Local\Temp\ipykernel_432\178818474.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  color[i] = delete_comma_in_title(color[i])


Сделаем таблицу, состоящую из колонок года и CPI для расчёта валовой и бюджетной стоимости фильмов с учётом инфляции

In [7]:
import requests
from bs4 import BeautifulSoup
url = 'https://www.usinflationcalculator.com/inflation/consumer-price-index-and-annual-percent-changes-from-1913-to-2008/'

r = requests.get(url)
data = r.text
soup = BeautifulSoup(data, 'html.parser')

# находим таблицу
table = soup.find('table')
rows = table.tbody.findAll('tr')

years = []
cpis = []
# строим таблицу, состоящую из колонок 'год - средняя cpi'
for row in rows:
    year = row.findAll('td')[0].get_text()
    if year.isdigit() and int(year) < 2017:
        years.append(int(year))
        cpis.append(float(row.findAll('td')[13].get_text()))

cpi_table = pd.DataFrame(
    {
        'year': years,
        'avg_annual_cpi': cpis
    }
)
cpi_table.head()

,year,avg_annual_cpi
0,1913,9.9
1,1914,10.0
2,1915,10.1
3,1916,10.9
4,1917,12.8


Считаем валовую и бюджетную стоимость фильмов

In [8]:
# # заменяем пропущенные значения на нули
df = df.fillna(0)

# функция, пересчитывающая старые доллары на новые
def get_real_value(nominal_amt, old_cpi, new_cpi):
    if nominal_amt and not old_cpi.empty:
        return float(nominal_amt) * new_cpi / float(old_cpi)
    else:
        return 0


CPI_2016 = float(cpi_table[cpi_table['year'] == 2016]['avg_annual_cpi'])

real_budget_values = []
real_gross_values = []

df['title_year'] = df['title_year'].replace('', 0)

# считаем реальную стоимость для каждой строки
for index, row in df.iterrows():
    gross = row['gross']
    budget = row['budget']
    year = row['title_year']

    cpi = cpi_table[cpi_table['year'] == int(year)]['avg_annual_cpi']

    real_gross = get_real_value(gross, cpi, CPI_2016)
    real_gross_values.append(real_gross)
    
    real_budget = get_real_value(budget, cpi, CPI_2016)
    real_budget_values.append(real_budget)

df['real_gross'] = real_gross_values
df['real_budget'] = real_budget_values

C:\Users\Azerty\AppData\Local\Temp\ipykernel_432\422668251.py:12: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  CPI_2016 = float(cpi_table[cpi_table['year'] == 2016]['avg_annual_cpi'])
C:\Users\Azerty\AppData\Local\Temp\ipykernel_432\422668251.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(nominal_amt) * new_cpi / float(old_cpi)


In [9]:
# экспорт таблицы в csv файл
df.to_csv('Lab1_Portnyagin_CPI.csv')

# Этап 3 - Проверка гипотез

## Гипотеза 1 - Период нулевых годов 21-го века является самым прибыльным для фильмов

### Чтобы проверить гипотезу 1, нужно
1. посчитать выручку, которая равна разности прибыли и бюджета фильмов
2. сгруппировать года по десятилетиям (10е, 20е, 30е года  т.д)
3. Вывести года по убыванию прибыли
4. Посмотреть, на каком месте находятся нулевые года (с 2000 по 2010)

### Считаем выручку по всем годам

In [10]:
# приводим года к одинаковому типу int16
df['title_year'] = df['title_year'].astype('int16')
# группируем фильмы по году и считаем для каждого года выручку и бюджет
profit_table = df.groupby('title_year')[['real_gross','real_budget']].sum()
# переводим числа в миллиарды долларов
profit_table /= 10**9
# считаем прибыль: выручка минус бюджет
profit_table['profit'] = profit_table['real_gross'] - profit_table['real_budget'] 
# сортируем прибыль по убыванию
profit_table = profit_table.sort_values(by='profit',ascending=False)
# выводим первую десятку самых прибыльных годов
profit_table[:10]

,real_gross,real_budget,profit
title_year,,,
1939,3.813485,0.142916,3.670568
1937,3.082181,0.033334,3.048847
2002,11.477607,8.674271,2.803336
2012,10.887788,8.145783,2.742005
1940,2.754937,0.133410,2.621528
1994,5.098076,2.490324,2.607753
1973,2.781690,0.193064,2.588626
2003,10.106006,7.571372,2.534634
2014,10.361783,8.017882,2.343901


### Группируем года по десятилетиям (нулевые, 10е, 20е и тд)
Из нулевых годов только 2002 и 2003 входят в топ самых прибыльных. Чтобы определить самые прибыльные десятилетия, нужно сгруппировать таблицу по 20м, 30м, 40м и т.д годам и сложить все прибыли годов этих десятилетий

In [11]:
# лист с десятилетиями (decades)
years = np.array(range(1920,2016,10))

# таблица в которой будет ключём - десятилетие, а значение - сумма прибыли годов этого десятилетия
decades_table = pd.Series()
# проходимся по десятилетиям
for i in range(len(years) - 1):
    # строки, подходящие под нужное десятилитие
    decade = profit_table[(years[i] < profit_table.index) & (profit_table.index <  years[i + 1])]
    # заносим в таблицу сумму прибылей годов данного десятилетия
    decades_table[f'{years[i]}-{years[i+1]}'] = round(decade['profit'].sum(),2)
# сортируем по убыванию
decades_table = decades_table.sort_values(ascending=False)
decades_table

1980-1990    12.60
1970-1980    12.30
1990-2000     8.94
1930-1940     6.67
1960-1970     4.92
1940-1950     1.58
1950-1960     0.57
1920-1930    -0.05
2000-2010   -10.59
dtype: float64

### **Вывод**
Нулевые года являются самыми худшими по прибыли с убытком 10.59 млрд долларов на пересчёт в деньги 2016 года. Первую позицию по заработку занимают 80-е - 90-е года с прибылью 12.6 млрд долларов. Возможно, результат нулевых годов такой неудачный из-за пропусков значений в столбце выручки (gross)

## Гипотеза 2 - Не существует таких жанров в США, где сумма лайков в фейсбуке 1ых актёров меньше суммы лайков 2ых или 3их актёров 

### Пример суммы лайков первых актёров жанра:
в трёх фильмах в жанре 'Action' первые актёры набрали 10, 20 и 30 лайков. Значит сумма лайков всех первых актёров жанра 'Action' равна 10+20+30=60

### Чтобы проверить гипотезу 2, нужно:
1. Отобрать только фильмы, сделанные в США
2. Разделить столбец c жанрами на разные столбцы
3. Посчитать суммы лайков актёров по жанрам
4. Вывести столбцы, где сумма лайков 1ых актёров < суммы лайков 2ых или 3их актёров
5. Если таких столбцов не оказалось, то гипотеза подтверждена

### Отбираем фильмы, которые сделаны в США и оставляем только столбцы лайков актёров и жанры

In [12]:
# заменяем пустые значения и приводим столбцы к одному типу int32
df[['Actor_1_Facebook_likes','actor_2_facebook_likes','actor_3_Facebook_likes']] = \
    df[['Actor_1_Facebook_likes','actor_2_facebook_likes','actor_3_Facebook_likes']].replace('',0).astype('int32')

# отбираем только строчки со страной США
df_USA = df.loc[df.loc[:,'country'] == 'USA']
# оставляем только столбцы с жанром и лайками 3 актёров 
df_USA = df_USA[['genres','Actor_1_Facebook_likes','actor_2_facebook_likes','actor_3_Facebook_likes']]
# выводим первые 5 строк таблицы
df_USA.head(5)

,genres,Actor_1_Facebook_likes,actor_2_facebook_likes,actor_3_Facebook_likes
0,Action|Adventure|Fantasy|Sci-Fi,1000,936,855
1,Action|Adventure|Fantasy,40000,5000,1000
3,Action|Thriller,27000,23000,23000
5,Action|Adventure|Sci-Fi,640,632,530
6,Action|Adventure|Romance,24000,11000,4000


### Разделяем столбцы с жанрами на разные столбцы

In [13]:
# функция, которая из одного столбца делает несколько и прибавляет их к датафрейму
def split_and_add_colums(df,column_to_split,split_sign):
    # разделяем строку на несколько строк и создаём из них столбцы
    df_where_split_column = df[column_to_split].str.split(split_sign, expand=True)
    # переименовываем названия столбцов
    df_where_split_column = df_where_split_column.rename(lambda x: f'{column_to_split}_{x}', axis=1)
    #соединяем датафреймы
    df = pd.concat([df,df_where_split_column],axis=1)
    # удаляем ненужный столбец
    df = df.drop(columns=column_to_split)
    # возвращаем новый датафрейм
    return df

df_USA = split_and_add_colums(df_USA,'genres','|')
df_USA.head(5)

,Actor_1_Facebook_likes,actor_2_facebook_likes,actor_3_Facebook_likes,genres_0,genres_1,genres_2,genres_3,genres_4,genres_5,genres_6,genres_7
0,1000,936,855,Action,Adventure,Fantasy,Sci-Fi,None,None,None,None
1,40000,5000,1000,Action,Adventure,Fantasy,None,None,None,None,None
3,27000,23000,23000,Action,Thriller,None,None,None,None,None,None
5,640,632,530,Action,Adventure,Sci-Fi,None,None,None,None,None
6,24000,11000,4000,Action,Adventure,Romance,None,None,None,None,None


### Считаем суммы лайков актёров по жанрам

In [14]:
# находим уникальные жанры из всех столбцов, содержащих жанры
unique_genres = pd.unique(df_USA.iloc[:,3:11].values.ravel())

# таблица, содержащая ключ - жанр, а значение - сумма по лайкам у трёх актёров фильмов
table_likes = pd.DataFrame()
# проходимся по жанрам
for genre in unique_genres:
    # берём строки, содержащие определённый жанр
    lines_containes_genre = df_USA[(df_USA == genre).any(axis=1)]
    # считаем сумму лайков актёров по этим жанрам
    summ_likes_actors = lines_containes_genre.iloc[:,0:3].sum()
    # добавляем в таблицу жанр и сумму лайков актёров
    table_likes[genre] = summ_likes_actors

# транспонируем таблицу для удобства
table_likes = pd.DataFrame.transpose(table_likes)
# выводим таблицу
table_likes.head(5)

,Actor_1_Facebook_likes,actor_2_facebook_likes,actor_3_Facebook_likes
Action,7921429.0,2088710.0,802663.0
Adventure,6142958.0,1889706.0,807455.0
Fantasy,4016419.0,1135803.0,552234.0
Sci-Fi,3603724.0,1069397.0,430652.0
None,0.0,0.0,0.0


### Выводим столцы, где сумма лайков 1ых актёров < суммы лайков 2ых или 3их

In [15]:
# 1 актёры
first_actors_likes = table_likes['Actor_1_Facebook_likes']
# 2 актёры
second_actors_likes = table_likes['actor_2_facebook_likes']
# 3 актёры
third_actors_likes = table_likes['actor_3_Facebook_likes']
# выведем строки, в которых лайки 1 актёров меньше, чем у вторых актёров
table_likes.loc[first_actors_likes < second_actors_likes]

,Actor_1_Facebook_likes,actor_2_facebook_likes,actor_3_Facebook_likes


In [16]:
# выведем строки, в которых лайки 1 актёров меньше, чем у третьих актёров
table_likes.loc[first_actors_likes < second_actors_likes]

,Actor_1_Facebook_likes,actor_2_facebook_likes,actor_3_Facebook_likes


In [17]:
# выведем строки, где лайки первых актёров менше суммы лайков 2ых и 3их актёров
table_likes.loc[first_actors_likes < (second_actors_likes + third_actors_likes)]

,Actor_1_Facebook_likes,actor_2_facebook_likes,actor_3_Facebook_likes
News,1056.0,937.0,190.0
Short,1698.0,1170.0,602.0


### **Вывод**
Как видим, не существует жанров, где суммма лайков первых актёров меньше, чем у вторых или третьих, следовательно гипотеза подтвердилась. Но нашлись жанры, где сумма лайков 1ых актёров меньше суммы лайков 2ых и 3их вместе взятых - новости и спорт

## Гипотеза 3 - Самые прибыльные жанры в США и в остальном мире совпадают 

### Чтобы проверить эту гипотезу, нам нужно:
1. разделить фильмы на 2 категории - сделанные и не сделанные в США
2. получить выручки фильмов
3. получить сумму выручек фильмов по разным жанрам 
4. вывести первые 10 фильмов по убыванию выручки
5. сравнить топ-10 фильмов США и остальных стран 

### Получаем фильмы, сделанные не в США

In [18]:
# берём строки, не содержащие США как страну
df_world = df.loc[df.loc[:,'country'] != 'USA']
# берём только столбцы с жанром, выручкой и бюджетом
df_world = df_world[['genres','real_budget','real_gross']]
df_world.head(5)

,genres,real_budget,real_gross
2,Action|Adventure|Thriller,2.480907e+08,2.025981e+08
4,Documentary,0.000000e+00,0.000000e+00
9,Adventure|Family|Fantasy|Mystery,2.796802e+08,3.378055e+08
12,Action|Adventure,2.229481e+08,1.876871e+08
20,Adventure|Fantasy,2.534543e+08,2.586332e+08


### Считаем выручки фильмов не из США

In [19]:
# считаем выручку
df_world['profit'] = df_world['real_gross'] - df_world['real_budget']
# берём только строки с положительной выручкой
df_world = df_world.loc[df_world.loc[:,'profit'] > 0]
# переводим в млн долларов
df_world['profit'] = round(df_world['profit'] / 10**6,ndigits=2)
#переименовываем столбец profit
df_world = df_world.rename(columns={'profit': 'profit (million $)'})
df_world.head(5)

,genres,real_budget,real_gross,profit (million $)
9,Adventure|Family|Fantasy|Mystery,2.796802e+08,3.378055e+08,58.13
20,Adventure|Fantasy,2.534543e+08,2.586332e+08,5.18
25,Action|Adventure|Drama|Romance,2.543853e+08,2.679664e+08,13.58
30,Action|Adventure|Thriller,2.090708e+08,3.181642e+08,109.09
34,Action|Adventure|Fantasy|Sci-Fi|Thriller,2.500073e+08,2.790082e+08,29.00


### Разделяем жанры по разным столбцам

In [20]:
# разделяем жанры о разным столбцам
df_world = split_and_add_colums(df_world,'genres','|')
# удаляем столбцы бюджета и выручки
df_world = df_world.drop(['real_budget','real_gross'],axis=1)
df_world.head(5)

,profit (million $),genres_0,genres_1,genres_2,genres_3,genres_4,genres_5,genres_6
9,58.13,Adventure,Family,Fantasy,Mystery,None,None,None
20,5.18,Adventure,Fantasy,None,None,None,None,None
25,13.58,Action,Adventure,Drama,Romance,None,None,None
30,109.09,Action,Adventure,Thriller,None,None,None,None
34,29.00,Action,Adventure,Fantasy,Sci-Fi,Thriller,None,None


### Считаем для каждого жанра сумму вырочек фильмов этого жанра

In [21]:
# функция, которая возвращает сумму бюджетов фильмов по определённому жанру
def genre_and_profit(df):
    # таблица, содержащая ключ - жанр, а значение - сумма выручки фильмов
    genres_profit = pd.Series()
    for genre in unique_genres:
        # берём строки, содержащие определённый жанр
        lines_containes_genre = df[(df == genre).any(axis=1)]
        # считаем сумму выручки по этим жанрам
        summ_profit_of_genre = lines_containes_genre['profit (million $)'].sum()
        # добавляем в таблицу жанр и сумму выручки
        genres_profit[genre] = summ_profit_of_genre
    return genres_profit.sort_values(ascending=False)

# выводим таблицу с жанрами и суммой выручки стран 
world_genres_profit = genre_and_profit(df_world)
world_genres_profit[:10]

Adventure    7076.13
Drama        5368.60
Action       5341.49
Thriller     5023.29
Comedy       3654.81
Family       2795.64
Fantasy      2601.45
Romance      2387.63
Sci-Fi       1968.79
Mystery      1582.51
dtype: float64

### Получаем выручки фильмов США

In [22]:
# позаимствуем таблицу df_USA из гипотезы 2
df_USA.head(5)

,Actor_1_Facebook_likes,actor_2_facebook_likes,actor_3_Facebook_likes,genres_0,genres_1,genres_2,genres_3,genres_4,genres_5,genres_6,genres_7
0,1000,936,855,Action,Adventure,Fantasy,Sci-Fi,None,None,None,None
1,40000,5000,1000,Action,Adventure,Fantasy,None,None,None,None,None
3,27000,23000,23000,Action,Thriller,None,None,None,None,None,None
5,640,632,530,Action,Adventure,Sci-Fi,None,None,None,None,None
6,24000,11000,4000,Action,Adventure,Romance,None,None,None,None,None


In [23]:
# удаляем колонки с лайками актёров
df_USA = df_USA.drop(df_USA.iloc[:,0:3], axis=1)
# получаем столбец выручки и бюджета фильмов США
gross_and_budget_USA = df.loc[df.loc[:,'country'] == 'USA'][['real_budget','real_gross']]
# переводим в миллионы долларов
gross_and_budget_USA = gross_and_budget_USA / 10**6
gross_and_budget_USA.head(5)

,real_budget,real_gross
0,265.136825,850.793694
1,347.332851,358.220754
3,261.338493,468.455147
5,275.659843,76.372180
6,298.706252,389.626765


In [24]:
# получаем выручку фильмов США
profit_USA = gross_and_budget_USA['real_gross'] - gross_and_budget_USA['real_budget']
profit_USA.head(5)

0    585.656870
1     10.887903
3    207.116654
5   -199.287662
6     90.920513
dtype: float64

### Считаем выручки фильмов США по жанрам

In [25]:
# соединяем с жанрами фильмов США
df_USA['profit (million $)'] = profit_USA
# берём только положительную выручку
df_USA = df_USA.loc[df_USA.loc[:,'profit (million $)'] > 0]
# округляем до 2х знаков после запятой
df_USA['profit (million $)'] = round(df_USA['profit (million $)'],2)
df_USA.head(5)

,genres_0,genres_1,genres_2,genres_3,genres_4,genres_5,genres_6,genres_7,profit (million $)
0,Action,Adventure,Fantasy,Sci-Fi,None,None,None,None,585.66
1,Action,Adventure,Fantasy,None,None,None,None,None,10.89
3,Action,Thriller,None,None,None,None,None,None,207.12
6,Action,Adventure,Romance,None,None,None,None,None,90.92
8,Action,Adventure,Sci-Fi,None,None,None,None,None,211.63


In [26]:
# получаем выручки фильмов жанров США
USA_genres_profit = genre_and_profit(df_USA)

In [27]:
# Выводим первую десятку жанров мира и США
print(world_genres_profit[:10])
print()
print(USA_genres_profit[:10])

Adventure    7076.13
Drama        5368.60
Action       5341.49
Thriller     5023.29
Comedy       3654.81
Family       2795.64
Fantasy      2601.45
Romance      2387.63
Sci-Fi       1968.79
Mystery      1582.51
dtype: float64

Drama        53515.13
Comedy       49040.97
Adventure    38482.23
Action       33509.89
Romance      30682.30
Family       30342.04
Fantasy      29609.28
Thriller     24941.24
Sci-Fi       21748.10
Crime        16891.92
dtype: float64


### **Вывод**
Как можно заметить, 9 из 10 выручек жанров мира и США совпадают, то есть находятся в топ-10. Следовательно, можно сделать предположение, что в США и мире одинаковые популярные жанры. Поэтому третья гипотеза подтвердилась

# Заключение

### Было проверено 3 гипотезы

1. Период нулевых годов 21-го века является самым прибыльным для фильмов
2. Не существует таких жанров в США, где сумма лайков в фейсбуке 1ых актёров меньше суммы лайков 2ых или 3их актёров 
3. Самые прибыльные жанры в США и в остальном мире совпадают

### Первая гипотеза опроверглась
Нулевые года 21 века занимают последнюю строчку по прибыльности фильмов, а не первую, как предполагалось

### Вторая Гипотеза подтвердилась
В действительности нет таких жанров, где сумма лайков 2ых или 3их актёров больше, чем у первых. Но есть жанры новости и спорт, где сумма лайков 1ых актёров не смогла превзойти сумму 2ых и 3их актёров вместе взятых

### Третья гипотеза подтвердилась
Самые прибыльные жанры в США и мире совпадают, только занимают разные места в топе 10 самых прибыльных жанров, а также есть различие в последних местах: 'Mystery' в мире и 'Crime' в США